In [1]:
import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
# os.environ['LANGCHAIN_API_KEY'] = <your api key>

os.environ['OPENAI_API_BASE'] = 'http://192.168.1.7:1234/v1/'
os.environ['OPENAI_API_KEY'] = 'lm-studio'

### Load documents

In [8]:
from pathlib import Path

from langchain_community.document_loaders import TextLoader


data_path = Path("vs_data")

documents = [
    TextLoader(doc_path).load()[0]
    for doc_path in data_path.iterdir()
]

### Split documents

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300, 
    chunk_overlap=50)

splits = text_splitter.split_documents(documents)

### Index

In [13]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=OpenAIEmbeddings(model='nomic-ai/nomic-embed-text-v1.5-GGUF', check_embedding_ctx_length=False, timeout=0.1, max_retries=50))

retriever = vectorstore.as_retriever()

### Create chain

In [16]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough

template = """Answer the question based on the context below. If the
question cannot be answered using the information provided answer
with "I don't know".

Context: {context}

Question: {question}

Answer: """

prompt = ChatPromptTemplate.from_template(template)

llm = ChatOpenAI(temperature=0)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, 
     "question": RunnablePassthrough()} 
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is the evolution of Lightning ring and how do i get it?")

'The evolution of Lightning Ring is Thunder Loop. To get Thunder Loop, you need to evolve Lightning Ring with Duplicator.'